In [1]:
import pandas as pd
import os.path as op
import retro
from src.features.annotations import generate_key_events, generate_aps_events, plot_bidsevents
from src.features.features import compute_framewise_aps
import matplotlib.pyplot as plt
from src.params import figures_path
from nilearn import plotting
from nilearn import image
import os
import numpy as np
from nilearn.plotting import plot_design_matrix
import load_confounds
from nilearn.input_data import NiftiMasker
from nilearn.glm.first_level import FirstLevelModel
import pickle
import nilearn

# This is a modified version of src.data.data.retrieve_variables, adapted to the naming of scan-related behavioural files
def retrieve_variables(files):
    '''
    files : list of files with complete path

    variable_lists : dictionnary (each variable is an entry) containing list of arrays of
    length corresponding to the number of frames in each run,
    with runs ordered by timestamp.
    '''

    variables_lists = {}

    for file in files:
        level = file[-11:-8]
        timestamp = file[-73:-65]
        print(file)
        if level == '5-0':
            env = retro.make('ShinobiIIIReturnOfTheNinjaMaster-Genesis', state='Level5')
        else:
            env = retro.make('ShinobiIIIReturnOfTheNinjaMaster-Genesis', state='Level'+level)
        actions = env.buttons

        run_variables = {}
        key_log = retro.Movie(file)
        env.reset()
        run_completed = False
        while key_log.step():
            a = [key_log.get_key(i, 0) for i in range(env.num_buttons)]
            _,_,done,i = env.step(a)

            if variables_lists == {}: # init final dict
                variables_lists['filename'] = []
                variables_lists['timestamp'] = []
                variables_lists['level'] = []
                for action in actions:
                    variables_lists[action] = []
                for variable in i.keys():
                    variables_lists[variable] = []

            if run_variables == {}: # init temp dict
                for variable in i.keys():
                    run_variables[variable] = []
                for action in actions:
                    run_variables[action] = []

            for variable in i.keys(): # fill up temp dict
                run_variables[variable].append(i[variable])
            for idx_a, action in enumerate(actions):
                run_variables[action].append(a[idx_a])

            if done == True:
                run_completed = True
        variables_lists['filename'].append(file)
        variables_lists['timestamp'].append(timestamp)
        variables_lists['level'].append(level)

        for variable in run_variables.keys():
            variables_lists[variable].append(run_variables[variable])
        env.close()
    return variables_lists


# This will go in src.features.annotations
def create_runevents(runvars, startevents, actions, FS=60, min_dur=1, get_aps=True, get_actions=True):
    onset_reps = startevents['onset'].values.tolist()
    dur_reps = startevents['duration'].values.tolist()
    lvl_reps = [x[-11] for x in startevents['stim_file'].values.tolist()]
    
    if get_aps:
        framewise_aps = compute_framewise_aps(runvars, actions=actions, FS=FS)

    # init df list
    all_df = []

    for idx, onset_rep in enumerate(onset_reps):
        print('Extracting events for {}'.format(runvars['filename'][idx]))
        if get_actions:
            # get the different possible actions
            # generate events for each of them
            for act in actions:
                var = runvars[act][idx]
                temp_df = generate_key_events(var, act, FS=FS)
                temp_df['onset'] = temp_df['onset'] + onset_rep
                temp_df['trial_type'] = lvl_reps[idx] + '_' + temp_df['trial_type']
                all_df.append(temp_df)
        if get_aps:
            temp_df = generate_aps_events(framewise_aps[idx], FS=FS)
            temp_df['onset'] = temp_df['onset'] + onset_rep
            temp_df['trial_type'] = lvl_reps[idx] + '_' + temp_df['trial_type']
            all_df.append(temp_df)

    events_df = pd.concat(all_df).sort_values(by='onset').reset_index(drop=True)
    return events_df



In [5]:


# Set constants for MULTIPLE RUNS
actions = ['B', 'A', 'MODE', 'START', 'UP', 'DOWN', 'LEFT', 'RIGHT', 'C', 'Y', 'X', 'Z']
dpath = '/media/hyruuk/Seagate Expansion Drive/DATA/shinobi/'
sub = 'sub-01'

testruns = ['1', '2']

seslist= os.listdir(dpath + sub)
for ses in ['ses-001']:#sorted(seslist):
    runs = [filename[-13] for filename in os.listdir(dpath + '{}/{}/func'.format(sub, ses)) if 'bold.nii.gz' in filename]
    allruns_events = []
    fmri_imgs = []
    for run in testruns:#sorted(runs):
        print('computing run {}'.format(run))
        events_fname = dpath + '{}/{}/func/{}_{}_task-shinobi_run-0{}_events.tsv'.format(sub, ses, sub, ses, run)
        filename = dpath + 'derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/sub-01_{}_task-shinobi_run-{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(sub, ses, ses, run)
        confounds_fname = dpath + 'derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/sub-01_{}_task-shinobi_run-{}_desc-confounds_timeseries.tsv'.format(sub, ses, ses, run)
        # Obtain list of bk2 files from events
        startevents = pd.read_table(events_fname)
        files = startevents['stim_file'].values.tolist()
        files = [dpath + file for file in files]

        # Retrieve variables from these files
        runvars = retrieve_variables(files)
        events_df = create_runevents(runvars, startevents, actions=actions)
        events_df['trial_type'].unique()

        # Create LvR_df
        lh_df = pd.concat([events_df[events_df['trial_type'] == '1_LEFT'], 
                           events_df[events_df['trial_type'] == '1_RIGHT'],
                           events_df[events_df['trial_type'] == '1_DOWN'],
                           events_df[events_df['trial_type'] == '1_UP'],
                           events_df[events_df['trial_type'] == '4_LEFT'], 
                           events_df[events_df['trial_type'] == '4_RIGHT'],
                           events_df[events_df['trial_type'] == '4_DOWN'],
                           events_df[events_df['trial_type'] == '4_UP'],
                            events_df[events_df['trial_type'] == '5_LEFT'], 
                           events_df[events_df['trial_type'] == '5_RIGHT'],
                           events_df[events_df['trial_type'] == '5_DOWN'],
                           events_df[events_df['trial_type'] == '5_UP']
                          ]).sort_values(by='onset').reset_index(drop=True)
        lh_df['trial_type'] = 'LeftH'
        rh_df = pd.concat([events_df[events_df['trial_type'] == '1_B'], 
                           events_df[events_df['trial_type'] == '1_C'],
                           events_df[events_df['trial_type'] == '4_B'], 
                           events_df[events_df['trial_type'] == '4_C'],
                            events_df[events_df['trial_type'] == '5_B'], 
                           events_df[events_df['trial_type'] == '5_C']
                          ]).sort_values(by='onset').reset_index(drop=True)
        rh_df['trial_type'] = 'RightH'
        LvR_df = pd.concat([lh_df, rh_df]).sort_values(by='onset').reset_index(drop=True)

        allruns_events.append(LvR_df)
        fmri_imgs.append(image.concat_imgs(filename))

        
confounds = []
confounds_cnames = []
for run in testruns:
    confounds_fname = dpath + 'derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/{}_{}_task-shinobi_run-{}_desc-confounds_timeseries.tsv'.format(sub, ses, sub, ses, run)

    confounds.append(pd.DataFrame.from_records(load_confounds.Params36().load(confounds_fname)))
    conf=load_confounds.Params36()
    conf.load(confounds_fname)
    #confounds.append(conf)
    confounds_cnames.append(conf.columns_)
        
design_matrices = []
for idx, run in enumerate(sorted(testruns)):
    t_r = 1.49
    n_slices = confounds[idx].shape[0]
    frame_times = np.arange(n_slices) * t_r

    design_matrix = nilearn.glm.first_level.make_first_level_design_matrix(frame_times,
                                                                           events=allruns_events[idx],
                                                                          drift_model=None,
                                                                          add_regs=confounds[idx],
                                                                          add_reg_names=confounds_cnames[idx])


    design_matrices.append(design_matrix)
    
fmri_glm = FirstLevelModel(t_r=1.49,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model=None, # reactivate automatic drift compensation
                           high_pass=.01,
                           n_jobs=-1,
                           smoothing_fwhm=5) # let's make this ￼simple and start without anatomical mask 



fmri_glm = fmri_glm.fit(fmri_imgs, design_matrices=design_matrices)
fmri_glm.generate_report(contrasts=['LeftH-RightH'])

computing run 1
/media/hyruuk/Seagate Expansion Drive/DATA/shinobi/sourcedata/sub-01/ses-shinobi_001/sub-01_ses-shinobi_001_20200914-150431_ShinobiIIIReturnOfTheNinjaMaster-Genesis_Level1-0_000.bk2
/media/hyruuk/Seagate Expansion Drive/DATA/shinobi/sourcedata/sub-01/ses-shinobi_001/sub-01_ses-shinobi_001_20200914-150431_ShinobiIIIReturnOfTheNinjaMaster-Genesis_Level4-1_000.bk2
/media/hyruuk/Seagate Expansion Drive/DATA/shinobi/sourcedata/sub-01/ses-shinobi_001/sub-01_ses-shinobi_001_20200914-150431_ShinobiIIIReturnOfTheNinjaMaster-Genesis_Level5-0_000.bk2
/media/hyruuk/Seagate Expansion Drive/DATA/shinobi/sourcedata/sub-01/ses-shinobi_001/sub-01_ses-shinobi_001_20200914-150431_ShinobiIIIReturnOfTheNinjaMaster-Genesis_Level1-0_001.bk2
/media/hyruuk/Seagate Expansion Drive/DATA/shinobi/sourcedata/sub-01/ses-shinobi_001/sub-01_ses-shinobi_001_20200914-150431_ShinobiIIIReturnOfTheNinjaMaster-Genesis_Level4-1_001.bk2
Extracting events for /media/hyruuk/Seagate Expansion Drive/DATA/shinobi/s

/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/nilearn/glm/first_level/first_level.py:580: UserWarning: One contrast given, assuming it for all 2 runs
  warn('One contrast given, assuming it for all %d runs' % n_runs)


ValueError: t contrasts should be length P=49, but this is length 47

In [ ]:
for conf in confounds:
    print(conf.shape)

In [ ]:
fmri_glm = FirstLevelModel(t_r=1.49,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model='cosine', # reactivate automatic drift compensation
                           high_pass=.01,
                           n_jobs=-1,
                           smoothing_fwhm=5) # let's make this ￼simple and start without anatomical mask 



fmri_glm = fmri_glm.fit(fmri_img, design_matrices=design_matrices)
fmri_glm.generate_report(contrasts=['LeftH-RightH'])